In [1]:
import os
import pickle

import matplotlib.pyplot as plt
import numpy
import torch

from splice import splice_model

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
sim_name = 'random_18.9k'
train = pickle.load(
        open(os.path.join("..", "data", "lgn-v1", sim_name, f"{sim_name}_train.pkl"), "rb")
    )
test = pickle.load(
    open(os.path.join("..", "data", "lgn-v1", sim_name, f"{sim_name}_test.pkl"), "rb")
)

lgn_train = torch.Tensor(train["responses"]["lgn"]).to(device)
v1_train = torch.Tensor(train["responses"]["v1"]).to(device)

lgn_test = torch.Tensor(test["responses"]["lgn"]).to(device)
v1_test = torch.Tensor(test["responses"]["v1"]).to(device)

In [5]:
model = splice_model(
    n_a = 400,
    n_b = 800,
    n_shared = 2,
    n_priv_a = 1,
    n_priv_b = 1,
    layers = [200,200,200,200,200,200],
    layers_msr = [200,200,200,200,200,200],
).to(device)

In [6]:
model.fit(lgn_train, v1_train, lgn_test, v1_test, disent_start=10000, c_disent=0.1, epochs=50000)

Epoch 0 	 source loss: 0.0540 | 0.0518 	 target loss: 0.0598 | 0.0574 	 disent loss: 1.0000 | 0.0000 	 msr loss: 0.0000 | 2.3973
Epoch 500 	 source loss: 0.0094 | 0.0096 	 target loss: 0.0112 | 0.0113 	 disent loss: 1.0000 | 278.6162 	 msr loss: 0.0000 | 309.3709
Epoch 1000 	 source loss: 0.0073 | 0.0078 	 target loss: 0.0085 | 0.0084 	 disent loss: 1.0000 | 272.5547 	 msr loss: 0.0000 | 306.6955
Epoch 1500 	 source loss: 0.0061 | 0.0064 	 target loss: 0.0071 | 0.0072 	 disent loss: 1.0000 | 309.1671 	 msr loss: 0.0000 | 349.3489
Epoch 2000 	 source loss: 0.0064 | 0.0067 	 target loss: 0.0071 | 0.0071 	 disent loss: 1.0000 | 88.2063 	 msr loss: 0.0000 | 103.7539
Epoch 2500 	 source loss: 0.0058 | 0.0061 	 target loss: 0.0052 | 0.0053 	 disent loss: 1.0000 | 111.6041 	 msr loss: 0.0000 | 130.1685
Epoch 3000 	 source loss: 0.0054 | 0.0058 	 target loss: 0.0036 | 0.0038 	 disent loss: 1.0000 | 121.1681 	 msr loss: 0.0000 | 141.4867
Epoch 3500 	 source loss: 0.0050 | 0.0054 	 target loss: 

In [14]:
torch.save(model.state_dict(), os.path.join("..", "results", "models", "lgn-v1", f"{sim_name}_splice.pt"))